# 使用 PyTorch 构建神经网络


In [1]:
from sklearn.datasets import load_iris

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import KFold
import sys

sys.path.append("../utils")
from _timer import Timer

# set random seed

torch.manual_seed(0)

In [18]:
# 加载数据
iris = load_iris()
X, y = iris.data, iris.target
X.shape, y.shape

((150, 4), (150,))

In [19]:
class Net(nn.Module):
    def __init__(
        self,
        num_inputs,
        num_outputs,
        verbose=False,
        epoch=5000,
        tol=1e-4,
        weight_decay=0.01,
    ):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_inputs, 10), nn.ReLU(), nn.Linear(10, num_outputs)
        )
        self.net.apply(self.init_weights)
        self.verbose = verbose
        self.epoch = epoch
        self.tol = tol
        self.weight_decay = weight_decay

    def init_weights(self, m):
        if type(m) == nn.Linear:
            nn.init.normal_(m.weight, std=0.01)

    def forward(self, x):
        return self.net(x)

    def cross_validate(self, X, y, k_folds=5):
        # 将数据转换为 PyTorch tensors
        X_tensor = torch.from_numpy(X).float()
        y_tensor = torch.from_numpy(y).long()
        accuracy = [0 for _ in range(k_folds)]

        # 创建一个 TensorDataset 对象
        dataset = TensorDataset(X_tensor, y_tensor)

        # 定义 k 折交叉验证
        kfold = KFold(n_splits=k_folds, shuffle=True)

        # 开始 k 折交叉验证
        for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
            # reset the network to default initialization
            self.net.apply(self.init_weights)
            train_size = len(train_ids)
            test_size = len(test_ids)

            # 创建训练和测试数据子集
            train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
            test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

            # 定义数据加载器
            train_loader = DataLoader(
                dataset, batch_size=train_size, sampler=train_subsampler
            )
            test_loader = DataLoader(
                dataset, batch_size=test_size, sampler=test_subsampler
            )
            # print(len(train_loader), len(test_loader))

            # 定义优化器和损失函数
            optimizer = optim.Adam(
                self.net.parameters(), weight_decay=self.weight_decay
            )
            criterion = nn.CrossEntropyLoss()

            # 训练网络
            epoch_loss_prev = -1
            for epoch in range(self.epoch):
                epoch_loss = 0
                for inputs, targets in train_loader:
                    optimizer.zero_grad()
                    outputs = self.net(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
                epoch_loss = epoch_loss / len(train_loader)
                if abs(epoch_loss_prev - epoch_loss) < self.tol:
                    print(f"terminate at Epoch {epoch+1}, Loss: {epoch_loss}")
                    break
                epoch_loss_prev = epoch_loss

                if self.verbose and epoch % 100 == 0:
                    print(f"Epoch {epoch+1}, Loss: {epoch_loss}")
                if epoch == self.epoch - 1:
                    print("Max iterations reached, stopping training...")

            # 在测试集上评估网络
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, targets in test_loader:
                    outputs = self.net(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    total += targets.size(0)
                    correct += (predicted == targets).sum().item()
            accuracy[fold] = correct / total
            print(f"Fold {fold}, Accuracy: {correct / total}")
        return accuracy

In [20]:
net = Net(X.shape[1], len(set(y)), verbose=0)

timer = Timer("neural network")


accuracy = net.cross_validate(X, y)
timer.stop()


average_accuracy = sum(accuracy) / len(accuracy)


print(f"Average accuracy: {average_accuracy}")

terminate at Epoch 1037, Loss: 0.12911881506443024
Fold 0, Accuracy: 1.0
terminate at Epoch 860, Loss: 0.09367472678422928
Fold 1, Accuracy: 0.9333333333333333
terminate at Epoch 958, Loss: 0.11977370828390121
Fold 2, Accuracy: 0.9666666666666667
terminate at Epoch 974, Loss: 0.1111510843038559
Fold 3, Accuracy: 0.9666666666666667
terminate at Epoch 1198, Loss: 0.1156303882598877
Fold 4, Accuracy: 1.0
Time taken for neural network: 8.69 seconds
Average accuracy: 0.9733333333333334


# 使用 scikit-learn MLPClassifier 构建神经网络


In [21]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier

# 加载数据
iris = load_iris()
X, y = iris.data, iris.target

timer = Timer("sklearn neural network")
# 创建神经网络模型
model = MLPClassifier(hidden_layer_sizes=(10,), learning_rate="adaptive", max_iter=3000)
timer.stop()

# 使用交叉验证评估模型
scores = cross_val_score(model, X, y, cv=5)

print("Cross-validation scores: ", *scores)
print("Average cross-validation score: ", scores.mean())

Time taken for sklearn neural network: 8.07e-05 seconds
Cross-validation scores:  1.0 1.0 0.9666666666666667 0.9666666666666667 1.0
Average cross-validation score:  0.9866666666666667
